# Process Lidar

This run was done with 8 m6a.4xlarge EC2 Instance.  Got them on that spot pricing dude!

In [1]:
import geopandas as gpd
import pandas as pd
#import geoplot as gplt
import laspy
import shapely
import os
from datetime import datetime, timedelta
import numpy as np
from itertools import product
import math
import pyproj
import re
import pyspark
import warnings
from collections import defaultdict

In [2]:
def lp(v, suppressOut=False):
    if not suppressOut:
        print(f"[{datetime.now()}] {v}")

lp("Starting...")

[2025-03-30 12:00:29.408309] Starting...


In [3]:
resolution = 1000  #grid resolution in meters
outPath = f"{os.curdir}{os.sep}output{os.sep}"
lazPath = f"{os.path.realpath(os.curdir)}{os.sep}datasets{os.sep}laz{os.sep}"
lp(f"lazPath:  {lazPath}")
assert os.path.exists(lazPath), f"{lazPath} does not exists PANIC!"

outputFileName = f"{outPath}AggregateLidarData_test_{resolution}m.parquet"
boxesFile = f"{outPath}boxesDf_{resolution}m.parquet"

captureHoods = []


[2025-03-30 12:00:29.413196] lazPath:  /home/g/Dev/NewOrleansElevation/datasets/laz/


In [4]:
executorRam = "8g"
conf = pyspark.SparkConf().setAppName(outputFileName).set("spark.executor.memory", executorRam).setMaster("spark://127.0.0.1:7077")
lp(conf)

[2025-03-30 12:00:29.419374] <pyspark.conf.SparkConf object at 0x7f7dbd696cd0>


In [5]:
if not os.path.exists(outPath):
    lp(f"Creating output path {outPath}")
    os.makedirs(outPath)

In [6]:
dataPath = f"{os.curdir}{os.sep}datasets{os.sep}"
fullDataPath = os.path.realpath(dataPath) + os.sep
lp(f"Path:  {dataPath}\t\tFullPath:{fullDataPath}")

[2025-03-30 12:00:29.427970] Path:  ./datasets/		FullPath:/home/g/Dev/NewOrleansElevation/datasets/


In [7]:
testLasFileName = os.listdir(f"{dataPath}laz")[5]
lp(f"Opening {testLasFileName} to get crs and more")
with laspy.open(f"{dataPath}laz{os.sep}{testLasFileName}", 'r') as f: 
    testLas = f.read()

[2025-03-30 12:00:29.432601] Opening USGS_LPC_LA_2021GreaterNewOrleans_C22_w0819n3342.laz to get crs and more


In [8]:
neighborhoodDf = gpd.read_file(f"{dataPath}Neighborhoods.geojson").to_crs(testLas.vlrs[0].parse_crs())

In [9]:
sorted(neighborhoodDf['gnocdc_lab'].unique())

['ALGIERS POINT',
 'AUDUBON',
 'B. W. COOPER',
 'BAYOU ST. JOHN',
 'BEHRMAN',
 'BLACK PEARL',
 'BROADMOOR',
 'BYWATER',
 'CENTRAL BUSINESS DISTRICT',
 'CENTRAL CITY',
 'CITY PARK',
 'DESIRE AREA',
 'DILLARD',
 'DIXON',
 'EAST CARROLLTON',
 'EAST RIVERSIDE',
 'FAIRGROUNDS',
 'FILMORE',
 'FISCHER DEV',
 'FLORIDA AREA',
 'FLORIDA DEV',
 'FRENCH QUARTER',
 'FRERET',
 'GARDEN DISTRICT',
 'GENTILLY TERRACE',
 'GENTILLY WOODS',
 'GERT TOWN',
 'HOLLYGROVE',
 'HOLY CROSS',
 'IBERVILLE',
 'IRISH CHANNEL',
 'LAKE CATHERINE',
 'LAKE TERRACE & OAKS',
 'LAKESHORE - LAKE VISTA',
 'LAKEVIEW',
 'LAKEWOOD',
 'LEONIDAS',
 'LITTLE WOODS',
 'LOWER GARDEN DISTRICT',
 'LOWER NINTH WARD',
 'MARIGNY',
 'MARLYVILLE - FONTAINEBLEAU',
 'MID-CITY',
 'MILAN',
 'MILNEBURG',
 'McDONOGH',
 'NAVARRE',
 'NEW AURORA - ENGLISH TURN',
 'OLD AURORA',
 'PINES VILLAGE',
 'PLUM ORCHARD',
 'PONTCHARTRAIN PARK',
 'READ BLVD EAST',
 'READ BLVD WEST',
 'SEVENTH WARD',
 'ST.  ANTHONY',
 'ST. BERNARD AREA',
 'ST. CLAUDE',
 'ST. ROCH

In [10]:
if len(captureHoods) > 0:
    captureHoodsMask = neighborhoodDf['gnocdc_lab'].isin(captureHoods)
else:
    captureHoodsMask = np.repeat(True, neighborhoodDf.shape[0])

In [11]:
bounds = [int(b) for b in neighborhoodDf[captureHoodsMask].total_bounds]

In [12]:
xPixels = (bounds[2] - bounds[0]) / resolution
yPixels = (bounds[3] - bounds[1]) / resolution
lp(f"Resolution will be {xPixels} x {yPixels}  Runtime based on {xPixels*yPixels}")

[2025-03-30 12:00:29.897817] Resolution will be 48.807 x 34.758  Runtime based on 1696.4337060000003


In [13]:
boxesDf = gpd.read_parquet(boxesFile)

FileNotFoundError: [Errno 2] Failed to open local file './output/boxesDf_1000m.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
lp("Creating spark context")
sc = pyspark.SparkContext(conf=conf)

In [ ]:
lp("Broadcasting boxes")
boxesBroadcast = sc.broadcast( np.array(list(boxesDf['geometry'].bounds.itertuples(name=None) )))

In [ ]:
boxesBroadcast.value[0]

In [ ]:
len(boxesBroadcast.value)

In [ ]:
tileIndex = gpd.read_file(f"{dataPath}USGS_LA_2021GNO_1_C22_TileIndex{os.sep}USGS_LA_2021GNO_1_C22_TileIndex.shp").to_crs(neighborhoodDf.crs)
tileIndex.index = tileIndex['Name'].map(lambda f: f"{fullDataPath}laz{os.sep}USGS_LPC_LA_2021GreaterNewOrleans_C22_{f}.laz")
tileIndexBroadcast = sc.broadcast(tileIndex['geometry'].map(lambda g: g.bounds).to_dict())
tileIndexBroadcast.value[tileIndex.index[0]]
lp(f"tileIndexBroadcast length:  {len(tileIndexBroadcast.value)}")

In [ ]:
def processLas(lasFileName, suppressOut=True):
    startLazTime = datetime.now()

    lp(f"Processing {lasFileName}...",suppressOut)
    
    with laspy.open(lasFileName, 'r') as f:
        las = f.read()

    

    outDict = {}

    lazBox = shapely.box(*tileIndexBroadcast.value[lasFileName])
    boxMask = gpd.GeoSeries(map(lambda b: shapely.box(*b[1:]), boxesBroadcast.value)).intersects(lazBox)
    
    for inputTup in boxesBroadcast.value[boxMask]:
        startTime = datetime.now()
        
        altTotal = 0
        waterTotal = 0
        pointTotal = 0
        intenseTotal = 0
    
    

        BoxIdx = inputTup[0]
        boxBounds = inputTup[1:]
            
        transBoxBounds = (  int((boxBounds[0] - las.header.offsets[0]) / las.header.scales[0]),\
                            int((boxBounds[1] - las.header.offsets[1]) / las.header.scales[1]),\
                            int((boxBounds[2] - las.header.offsets[0]) / las.header.scales[0]),\
                            int((boxBounds[3] - las.header.offsets[1]) / las.header.scales[1]),)

        
        lp(f"LAZ Bounds         = ({las.header.mins[0]},{las.header.mins[1]},{las.header.maxs[0]},{las.header.maxs[1]})",suppressOut)
        lp(f"LAZ Unscale Bounds = ({las.X.min()},{las.Y.min()},{las.X.max()},{las.y.max()})",suppressOut)
        lp(f"newBox:  {transBoxBounds}", suppressOut)
        
        X = las.X
        Y = las.Y
        Z = las.Z
        cls = las.classification
        intense = las.intensity

                
        groundMask = np.isin(cls, [2,9])
        inBoundsMaskX = np.logical_and(X >= (transBoxBounds[0]), (X <= (transBoxBounds[2])))
        lp(f"InBoundsX:  {np.count_nonzero(inBoundsMaskX)}", suppressOut)
        inBoundsMaskY = np.logical_and(Y >= (transBoxBounds[1]), (Y <= (transBoxBounds[3])))
        lp(f"InBoundsY:  {np.count_nonzero(inBoundsMaskY)}", suppressOut)
        inBoundsMask = np.logical_and(inBoundsMaskX,inBoundsMaskY)
        lp(f"InBounds:  {np.count_nonzero(inBoundsMask)}", suppressOut)
        goodPointMask = np.logical_and( groundMask, inBoundsMask)
        
        lp(f"Good points / Total {np.count_nonzero(goodPointMask)}/{goodPointMask.shape[0]}", suppressOut)
 
        altTotal += int(Z[goodPointMask].sum())
        waterTotal += np.count_nonzero(cls[goodPointMask] == 9)
        intenseTotal += int(intense[goodPointMask].sum())
        pointTotal += np.count_nonzero(goodPointMask)

        outDict[BoxIdx] = (altTotal, waterTotal, intenseTotal, pointTotal, datetime.now() - startTime)
        

    lp(f"LAZ:  {lasFileName} processed in {datetime.now()-startLazTime}")

    return (lasFileName, outDict)


In [ ]:
def seqOp(x1, x2):
    outDict = defaultdict(lambda: (0,0,0,0, timedelta(0)), x1)
    x2Dict = x2[1]
    
    for key, value in x2Dict.items():
        outDict[key] = tuple((outDict[key][i] + value[i] for i in range(5) ))

    return dict(outDict)

def comboOp(x1,x2):
    outDict = {}
    x1Dict =defaultdict(lambda: (0,0,0,0, timedelta(0)), x1)
    x2Dict =defaultdict(lambda: (0,0,0,0, timedelta(0)), x2)
    for boxTup in boxesBroadcast.value:
        idx = boxTup[0]
        outDict[idx] = tuple((x1Dict[idx][i] + x2Dict[idx][i] for i in range(5)))

    return outDict


#testing
if False:
    
    test1 = processLas(os.path.realpath("datasets/laz/USGS_LPC_LA_2021GreaterNewOrleans_C22_w0776n3315.laz"), suppressOut=False)
    test2 = processLas(os.path.realpath("datasets/laz/USGS_LPC_LA_2021GreaterNewOrleans_C22_w0780n3324.laz"), suppressOut=False)
    testSeq1 = seqOp({}, test1)   
    testSeq2 = seqOp(testSeq1, test2)
    print(testSeq2)
                     
    out = comboOp(comboOp(testSeq1, testSeq2), testSeq1)

    print(out)

In [ ]:
lazFiles = [lazPath + p for p in os.listdir(lazPath) if p[-4:]=='.laz'][:10]

In [ ]:
partitions = round(len(lazFiles) / 3)
lp(f"Partitions:  {partitions}")
boxesRdd = sc.parallelize(lazFiles, partitions)

In [ ]:
boxesProcessedRdd = boxesRdd.map(processLas)

In [ ]:
lp(f"{boxesDf.shape[0]} boxes total")

In [ ]:
sparkTime = datetime.now()
output = boxesProcessedRdd.aggregate({}, seqOp, comboOp)
lp(f"Spark completed in {datetime.now() - sparkTime}")

In [ ]:
sc.stop()

In [ ]:
#If those code were to fail after reteiving all of the reduced data
#we would be soooooo sad
#to wit this is a simplified output meathod by date
#which can be used to recreate the dataset if the unthinkable
failSafeOutFile = outPath + datetime.now().strftime('%Y-%m-%d_%H%M%S') + '.data'
lp(f"Writing failsafe data to {failSafeOutFile}")
with open(failSafeOutFile, 'w') as f:
    for i in range(len(output)):
        f.write(str(output[i]))
        f.write("\n")

lp(f"Done failsafe data to {failSafeOutFile}!")

In [ ]:
lp("Starting boxesDf load")
for key,value in output.items():

    boxesDf.loc[key, 'AltitudeTotal'] = value[0]
    boxesDf.loc[key, 'WaterTotal'] = value[1]
    boxesDf.loc[key, 'Int'] = value[2]
    boxesDf.loc[key, 'Total'] = value[3]
    boxesDf.loc[key, 'RunTime'] = value[4]

lp("Finished boxesDf load")

In [ ]:
boxesDf.head()

In [ ]:
lp(f"Sending boxesDf to {outputFileName}")
boxesDf.to_parquet( outputFileName )
lp(f"Done boxesDf to {outputFileName}")

In [ ]:
lp(f"Mean runtime:  {boxesDf['RunTime'].mean()}")
lp(f"Max runtime:  {boxesDf['RunTime'].max()}")
lp(f"Total runtime:  {boxesDf['RunTime'].sum()}")
lp(f"Totals Min/Mean/Max\t\t{boxesDf['Total'].min()} / {boxesDf['Total'].mean()} / {boxesDf['Total'].max()}")

In [ ]:
if True:
    boxesDf['AltCalc'] = boxesDf['AltitudeTotal'] / boxesDf['Total']
    boxesDf.loc[pd.isna(boxesDf['AltCalc']),'AltCalc'] = 0
    ax = neighborhoodDf.plot(color='pink')
    boxesDf.plot(column='AltCalc',ax=ax)

In [ ]:
if True:
    boxesDf['IntCalc'] = boxesDf['Int'] / boxesDf['Total']
    boxesDf.loc[pd.isna(boxesDf['IntCalc']),'IntCalc'] = 0
    ax = neighborhoodDf.plot(color='pink')
    boxesDf.plot(column='IntCalc',ax=ax)

In [ ]:
lp("Done!")

In [ ]:
#!aws s3 cp $outputFileName s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/ --dryrun